# Coordinate Conversion in ctapipe

As conversion between different coordinate systems is pretty much required before the event reconstruction can take place (as reconstruction typically takes place in a common reference system). I therefore took a relatively quick first pass at the kind of system we may want to use in the final implementation. The code is based on the astropy coordinates code:
http://docs.astropy.org/en/stable/coordinates/

The astropy scheme allows us define a number of coordinate frames and the tranformations between any 2 frames, the user then simply calls the transform to function and the astropy finds the route from one system to the next.

The tranformation code is based on:
- Konrad's read_hess code
- The H.E.S.S. software
- A small amount of independent thought

In [1]:
from ctapipe.coordinates import *
import numpy as np
import astropy.units as u

## Angular Systems
Our ultimate aim for direction reconstruction is to be able to convert from the XY position of pixels in the camera focal plane, to a shared nominal angular system for all telescopes in the array. This system is where event reconstruction can take place, then the reconstructed position can be further converted into an alt-az system (which can further be converted into any astronomical system).

In order to do this I created 3 coordinate reference frames.

### Camera Frame
Physical system describing the position of pixels within the camera focal plane, defined in distance units.
### Telescope Frame
Angular system decribing the XY offset of objects from the pointing direction of the instrument (assumed to be 0,0 in both camera and telescope frames). Defined in angular units.
### Nominal Frame
Angular system describing the XY offset of objects from the nomial array pointing direction (assumed to be 0,0 in this frame). Defined in angular units.

First lets take a look at camera/angular systems. First create a numpy array of x-y positions (equivelent of creating a camera pixel).

In [2]:
 pix = [1,2,0] #z positions set to zero (as in most cameras)

Then we pass this array to CameraFrame object

In [3]:
camera_coord = CameraFrame(pix*u.m) # has to be in distance units

We then can directly convert this to the nominal telescope system. Need to know telescope focal length and camera rotation (if any).

In [4]:
telescope_coord = camera_coord.transform_to(TelescopeFrame(focal_length = 15*u.m,rotation=0*u.deg,
                                                           pointing_direction = [70*u.deg,180*u.deg])) 
# pointing direction should maybe be changed to use astropy style AltAz object

print("Telescope Coordinate",telescope_coord)

Telescope Coordinate <TelescopeFrame Coordinate (focal_length=15.0 m, rotation=0.0 deg, pointing_direction=[<Quantity 70.0 deg>, <Quantity 180.0 deg>]): (x, y, z) in rad
    (0.06666667, 0.13333333, 0.0)>


In [5]:
print ("X coordinate",telescope_coord.x)

X coordinate 0.06666666666666667 rad


From Telescope system we can them then convert to array nominal pointing system

In [6]:
nominal_coord = telescope_coord.transform_to(NominalFrame(array_direction = [70*u.deg,180*u.deg]))
print ("Nominal Coordinate",nominal_coord)

Nominal Coordinate <NominalFrame Coordinate (array_direction=[<Quantity 70.0 deg>, <Quantity 180.0 deg>], pointing_direction=None, rotation=0.0 deg, focal_length=None): (x, y, z) in rad
    (0.06666667, 0.13333333, 0.0)>


In [7]:
nominal_coord = telescope_coord.transform_to(NominalFrame(array_direction = [71*u.deg,180*u.deg]))
print ("Nominal Coordinate",nominal_coord)

Nominal Coordinate <NominalFrame Coordinate (array_direction=[<Quantity 71.0 deg>, <Quantity 180.0 deg>], pointing_direction=None, rotation=0.0 deg, focal_length=None): (x, y, z) in rad
    (0.0491544, 0.13319864, 0.0)>


Or if we are feeling clever we can bypass the telescope system entirely.

In [8]:
nominal_coord = camera_coord.transform_to(NominalFrame(focal_length = 15*u.m,rotation=0*u.deg,
                                                        pointing_direction = [70*u.deg,180*u.deg],
                                                        array_direction = [71*u.deg,180*u.deg]))
print ("Nominal Coordinate",nominal_coord)

Nominal Coordinate <NominalFrame Coordinate (array_direction=[<Quantity 71.0 deg>, <Quantity 180.0 deg>], pointing_direction=[<Quantity 70.0 deg>, <Quantity 180.0 deg>], rotation=0.0 deg, focal_length=15.0 m): (x, y, z) in rad
    (0.0491544, 0.13319864, 0.0)>


Normally we would then want to go to the alt-az system. This ten puts us in the world of astropy coordinates and we can transform to any astronomical system, although in the pipeline this shouldn't be needed often.

In [9]:
from astropy.coordinates import AltAz
altaz_coord = nominal_coord.transform_to(AltAz)
print (altaz_coord)

<AltAz Coordinate (obstime=None, location=None, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    (205.51316175, 72.17100816)>


## Ground System
As well as angular systems we also often need to work in a ground based system for energy reconstruction. Only to systems are neccesary.

Ground system
- True XYZ coordinates on the ground

Tilted ground system
- XY system, projection of the ground system into an inclined plane
- Plane could be the telescope pointing direction (often used in HESS for event reconstruction)
- Alteratively could be the reconstructed shower direction (or anything else)

In [10]:
grd_coord = GroundFrame(x=100*u.m, y=0*u.m, z=0*u.m) # GroundFrame always defined in distance units
print (grd_coord)

<GroundFrame Coordinate (pointing_direction=None): (x, y, z) in m
    (100.0, 0.0, 0.0)>


In [11]:
tilt_coord = grd_coord.transform_to(TiltedGroundFrame(pointing_direction = [90*u.deg,180*u.deg]))
print (tilt_coord)

<TiltedGroundFrame Coordinate (pointing_direction=[<Quantity 90.0 deg>, <Quantity 180.0 deg>]): (x, y, z) in m
    (-100.0, 0.0, 0.0)>


In [12]:
tilt_coord = grd_coord.transform_to(TiltedGroundFrame(pointing_direction = [70*u.deg,0*u.deg]))
print (tilt_coord)

<TiltedGroundFrame Coordinate (pointing_direction=[<Quantity 70.0 deg>, <Quantity 0.0 deg>]): (x, y, z) in m
    (93.96926208, 0.0, 0.0)>


In [13]:
grd_coord = tilt_coord.transform_to(GroundFrame)
print (grd_coord)

<GroundFrame Coordinate (pointing_direction=None): (x, y, z) in m
    (88.30222216, 0.0, -32.13938048)>


Hmm so we don't get back to ouor original value. This is not unexpected due to the nature of the tilted system, this is now giving us the XYZ position of the point within the tilted frame, but in a ground based reference system (not really useful). More common usage would be to project the point in the tilted system back to where it would lie in the ground.

In [14]:
print(project_to_ground(tilt_coord))

<GroundFrame Coordinate (pointing_direction=None): (x, y, z) in m
    (100.0, 0.0, 0.0)>


Transforming objects from tilted to ground frames is not neccessarily something we want to do very often, but we should define exactly how we want these convertions to behave.

## Performance

Of course this is the kind of function we call alot, so we should be careful of the performance.

In [15]:
%%timeit
telescope_coord = camera_coord.transform_to(TelescopeFrame(focal_length = 15*u.m,rotation=0*u.deg,
                                                           pointing_direction = [70*u.deg,180*u.deg])) 


1000 loops, best of 3: 1.49 ms per loop


In [16]:
%%timeit
nominal_coord = camera_coord.transform_to(NominalFrame(focal_length = 15*u.m,rotation=0*u.deg,
                                                        pointing_direction = [70*u.deg,180*u.deg],
                                                        array_direction = [71*u.deg,180*u.deg]))

100 loops, best of 3: 3.28 ms per loop


In [17]:
%%timeit
tilt_coord = grd_coord.transform_to(TiltedGroundFrame(pointing_direction = [70*u.deg,0*u.deg]))

1000 loops, best of 3: 1.59 ms per loop


Currently the code is taking a few milliseconds to execute, but the actual execution time of the transformation within only takes a few microseconds, as it is essentially all just numpy operations, so most time must be taken up in checks and the determination of the "route" from one frame to another.

Millisecond execution time is likely not fast enough, although we can at least reduce the impact of these overheads by being a bit cleverer about how we perform the conversions. For example we can convert a whole camera at once.

In [18]:
x = np.ones(2048)
y = np.ones(2048)*2
z = np.zeros(2048)

whole_camera = CameraFrame(x=x*u.m,y=y*u.m,z=z*u.m)
print (whole_camera)

<CameraFrame Coordinate: (x, y, z) in m
    [(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
     (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)]>


In [19]:
%%timeit
nominal_coord = whole_camera.transform_to(NominalFrame(focal_length = 15*u.m,rotation=0*u.deg,
                                                        pointing_direction = [70*u.deg,180*u.deg],
                                                        array_direction = [71*u.deg,180*u.deg]))

100 loops, best of 3: 4.55 ms per loop


In [20]:
print (nominal_coord)

<NominalFrame Coordinate (array_direction=[<Quantity 71.0 deg>, <Quantity 180.0 deg>], pointing_direction=[<Quantity 70.0 deg>, <Quantity 180.0 deg>], rotation=0.0 deg, focal_length=15.0 m): (x, y, z) in rad
    (0.0491544, 0.13319864, 0.0)>


Or even we could convert a whole event at once (all telescopes in one go)

In [21]:
x = np.ones([2048,10]) # For example 10 telescopes each with 2048 pixels
y = np.ones([2048,10])*2
z = np.zeros([2048,10])

whole_event = CameraFrame(x=x*u.m,y=y*u.m,z=z*u.m)
print (whole_event)

<CameraFrame Coordinate: (x, y, z) in m
    [[(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
      (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)],
     [(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
      (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)],
     [(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
      (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)],
     ..., 
     [(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
      (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)],
     [(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
      (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)],
     [(1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), ...,
      (1.0, 2.0, 0.0), (1.0, 2.0, 0.0), (1.0, 2.0, 0.0)]]>


In [22]:
%%timeit
nominal_coord = whole_event.transform_to(NominalFrame(focal_length = 15*u.m,rotation=0*u.deg,
                                                        pointing_direction = [70*u.deg,180*u.deg],
                                                        array_direction = [71*u.deg,180*u.deg]))

100 loops, best of 3: 11 ms per loop


In [23]:
print (nominal_coord)

<NominalFrame Coordinate (array_direction=[<Quantity 71.0 deg>, <Quantity 180.0 deg>], pointing_direction=[<Quantity 70.0 deg>, <Quantity 180.0 deg>], rotation=0.0 deg, focal_length=15.0 m): (x, y, z) in rad
    (0.0491544, 0.13319864, 0.0)>


Now we are back in the range of being dominated by the actual conversion execution. But of course CTA will be made up of different telescope types with different focal lengths and potentially even looking at different places.

In [24]:
%%timeit

focal_length = [23,23,23,15,15,15,15,4,4,4] * u.m
pointing_direction = [[70.5,71,72,73,69,68,67,66,65,70],
                      [180,180,180,180,180,180,180,180,180,180]]*u.deg # probably should switch this to defining pairs
nominal_coord_event = whole_event.transform_to(NominalFrame(focal_length = focal_length,rotation=0*u.deg,
                                                        pointing_direction = pointing_direction,
                                                        array_direction = [71*u.deg,180*u.deg]))



100 loops, best of 3: 11.3 ms per loop


Certainly the conversion of the full camera (or all important pixels), makes sense but for whole events I'm unsure. The major problem here is the definition of the pixel array length, as this second diminsion of the numpy array cannot be varied. This especially gets more difficult when we are dealing with zero-suppressed data where the pixel list could be any length.

One option is to just define ther length of this dimension as the longest value in the pixel lists (or the cameras), but at some point we will have a 25,000 pixel SCT camera and this will get messy.

Maybe some python master has a solution to this problem.

## To do

- Code needs more thorugh check of accuracy, especially when using multi-telescope arrays
- Exceptions need to be handled in a more robust way
- Speed optimisation, for now this system is probably fast enough. But we may want to optimise performance (could be done by contibuting to astropy) in future or potentially switch to a different system.
- Currently nominal systems defined as 3d catesian systems, but would make sense for them to be only 2D
- Not certain conversions are correct in the case of large distances between observation positions
- Getting the pointing information (and corrections) into the different frames

In [ ]:
%matplotlib inline 

import matplotlib.pyplot as plt
from numpy import sqrt
from numpy import meshgrid
from numpy import arange

xs = arange(-7.25, 7.25, 0.01)
ys = arange(-5, 5, 0.01)
x, y = meshgrid(xs, ys)

eq1 = ((x/7)**2*sqrt(abs(abs(x)-3)/(abs(x)-3))+(y/3)**2*sqrt(abs(y+3/7*sqrt(33))/(y+3/7*sqrt(33)))-1)
eq2 = (abs(x/2)-((3*sqrt(33)-7)/112)*x**2-3+sqrt(1-(abs(abs(x)-2)-1)**2)-y)
eq3 = (9*sqrt(abs((abs(x)-1)*(abs(x)-.75))/((1-abs(x))*(abs(x)-.75)))-8*abs(x)-y)
eq4 = (3*abs(x)+.75*sqrt(abs((abs(x)-.75)*(abs(x)-.5))/((.75-abs(x))*(abs(x)-.5)))-y)
eq5 = (2.25*sqrt(abs((x-.5)*(x+.5))/((.5-x)*(.5+x)))-y)
eq6 = (6*sqrt(10)/7+(1.5-.5*abs(x))*sqrt(abs(abs(x)-1)/(abs(x)-1))-(6*sqrt(10)/14)*sqrt(4-(abs(x)-1)**2)-y)

#eq1 = ((x/7.0)**2.0*sqrt(abs(abs(x)-3.0)/(abs(x)-3.0))+(y/3.0)**2.0*sqrt(abs(y+3.0/7.0*sqrt(33.0))/(y+3.0/7.0*sqrt(33.0)))-1.0)

for f in [eq1,eq2,eq3,eq4,eq5,eq6]:
    plt.contour(x, y, f, [0])

plt.show()